In [ ]:
import networkx as nx
import os
import numpy as np
import pandas as pd

## 1. Data

### Clean the dataframes

We read the datasets from the `csv` files, then we clean them according to the required steps in the `readme` file describing the homework:
- Space at the end and slash at the end are removed in the names from `hero_network`.
- Rows inducing loops are removed.
- Solve the Spider-Man problem.

In [ ]:
# Read the dataframes
edges = pd.read_csv(os.path.join('data', 'edges.csv'))
hero_network = pd.read_csv(os.path.join('data', 'hero-network.csv'))
nodes = pd.read_csv(os.path.join('data', 'nodes.csv'))

In [ ]:
# This simple Regex substitution solves slash and space problem in hero network
hero_network['hero1'] = hero_network.hero1.str.replace(r'/$|\s$', '', regex=True)
hero_network['hero2'] = hero_network.hero2.str.replace(r'/$|\s$', '', regex=True)

# Remove rows inducing loops in the graph
hero_network = hero_network[~(hero_network.hero1 == hero_network.hero2)].copy()

In [ ]:
# Fix Peter Parker name
hero_network.loc[hero_network.hero1 == 'SPIDER-MAN/PETER PAR', 'hero1'] = 'SPIDER-MAN/PETER PARKER'
hero_network.loc[hero_network.hero2 == 'SPIDER-MAN/PETER PAR', 'hero2'] = 'SPIDER-MAN/PETER PARKER'

### Build the graphs

Let's build the first graph, the one from `hero_network`, undirected and weighted, with weights decreasing as the number of common appearances in comics increases. The approach chosen is similar to an idf measure, with the weight being given by $\frac{1}{\log(x)+1}$.

In [ ]:
# Count appearances of hero 1 and hero 2 (ordered)
hero_counts = hero_network.value_counts()
# Group with frozenset, this way we get count of appearances without taking into consideration the order
hero_counts = hero_counts.groupby(lambda x: frozenset(x)).sum()
# Move frozenset from index to column
hero_counts = hero_counts.reset_index(name = 'count')

# List constructor onto frozenset (because pd.Series requires order) and then pd.Series to split the lists and get two different series
# (resulting in a DataFrame). Then concatenate count Series with these other two we have obtained. This is close to the final dataset we need.
hero_counts = pd.concat([hero_counts['index'].apply(list).apply(pd.Series), hero_counts['count']], axis = 1)

hero_counts['weight'] = 1/(1+np.log(hero_counts['count'])) # Compute weights with something quite similar to idf weights
hero_counts.drop('count', axis = 1, inplace = True) # Drop count column, we don't need that anymore
hero_counts.rename({0: 'hero1', 1:'hero2'}, axis = 1, inplace = True) # Rename for tidiness

# Build the undirected graph from edgelist + weight attribute
hero_graph = nx.from_pandas_edgelist(hero_counts, source='hero1', target='hero2', edge_attr='weight')

Now we build the second graph, the one from edges and considering the node type. For this we need to use `nodes` and `edges`.

In [ ]:
hero_comics_graph = nx.from_pandas_edgelist(edges, source='hero', target='comic')
nodes_attr_dict = nodes.set_index('node').to_dict(orient='index')
nx.set_node_attributes(hero_comics_graph, nodes_attr_dict)

In [ ]:
import pickle
with open('graphs.pickle', 'wb') as file:
    pickle.dump([hero_graph, hero_comics_graph], file)